# 1. Algorithmic Thinking, Peak Finding
- **Problem: Find a peak in an array (if it exists).**
    - For ease of understanding, henceforth, O will be used to refer to Θ notation.
- In a 1D array of numbers, a position is a peak if and only if the number at that position is >= the numbers before and after it in the array.  
    - At the edges, you'll only need to look at the element to the right (for the first) or left (for the last) of the edge.  
- **Straightforward algorithm:**
    - Traverse array from beginning to end
    - Complexity is Θ(n)
- **Divide and conquer algorithm:**
    - In array of n positions, look at n/2 position
        - if a[n/2] < a[n/2 -1] then only look at the left half of the array (positions 1 -> n/2 -1 to find a peak)
        - else if a[n/2] < a[n/2 + 1] then only look at the right half of the array (positions n/2 + 1 -> n to find a peak)
        - else n/2 is a peak
    - Complexity is Θ(log<sub>2</sub>n)
        - T(n) = T(n/2) + Θ(1)
        - base case is Θ(1) --> in the case of a 1 element array
- Comparing these 2 algorithms:
    - divide and conquer algorithm is significantly faster than the straightforward one.
- In a 2D array of numbers, a position is a peak if and only if the number at teh position is >= the numbers above, below, to the left, and right of it.
    - For the purposes of discussion, n = number of rows, m = number of columns
- **Greedy Ascent "Algorithm" - 2D:**
    - Picks a direction and tries to follow a direction in order to find a peak.
    - Has to make choice as to where to start and move
    - Complexity is Θ(nm) or if n = m, Θ(n^2)
- **Divide and conquer algorithm (incorrect) - 2D**:
    - Pick middle column j = m/2
    - Find a 1D peak at in the j column at some position in the 2D array (i, j)
    - Use row (i) as a start to find a 1D peak on row (i)
    - This would be efficient (Θ(log<sub>2</sub>n) or Θ(log<sub>2</sub>m)) but it DOESN'T work.
        - Once you try and find a peak on row i, you may no longer have a peak at that column anymore. A 2D peak is not guaranteed.   
- **Divide and conquer algorithm (correct)**:
    - Pick a middle column j = m/2
    - Find global maximum on column j at some position (i, j)
    - Compare (i, j-1), (i, j), (i, j+1)
    - Pick left columns if (i, j-1) > (i, j) OR pick right columns if (i, j) > (i, j + 1)
        - This decreases the number of columns to deal with (half)
        - Solve the problem with half the number of columns
    - In the case that (i, j) >= (i, j-1) and (i, j) >= (i, j+1) --> (i, j) is a peak.
    - In the case of a single column, find the global maximum, and you're done.
    - Complexity is Θ(nlog<sub>2</sub>m)
        - T(n, m) = T(n, m/2) + Θ(n) where Θ(n) is the work to find the global maximum
        - Base case: T(n,1) = Θ(n)
- Note: all of these algorithms (written out below) are recursive.

In [ ]:
#Note: problem is a defined type with special methods that represents a peak finding problem.
# Trace is another defined type used in the visualizer provided.

# 2D Peak Finding Algorithm - Recursive 
# Complexity: O(nlog(n))

def algorithm1(problem, trace = None):
    # if it's empty, we're done 
    if problem.numRow <= 0 or problem.numCol <= 0:
        return None

    # the recursive subproblem will involve half the number of columns
    mid = problem.numCol // 2

    # information about the two subproblems
    (subStartR, subNumR) = (0, problem.numRow)
    (subStartC1, subNumC1) = (0, mid)
    (subStartC2, subNumC2) = (mid + 1, problem.numCol - (mid + 1))

    subproblems = []
    subproblems.append((subStartR, subStartC1, subNumR, subNumC1))
    subproblems.append((subStartR, subStartC2, subNumR, subNumC2))

    # get a list of all locations in the dividing column
    divider = crossProduct(range(problem.numRow), [mid])

    # find the maximum in the dividing column
    bestLoc = problem.getMaximum(divider, trace)

    # see if the maximum value we found on the dividing line has a better
    # neighbor (which cannot be on the dividing line, because we know that
    # this location is the best on the dividing line)
    #getBetterNeighbor returns the location of the larger neighbor relative to given position
    neighbor = problem.getBetterNeighbor(bestLoc, trace)
    

    # this is a peak, so return it
    if neighbor == bestLoc:
        if not trace is None: trace.foundPeak(bestLoc)
        return bestLoc
   
    # otherwise, figure out which subproblem contains the neighbor, and
    # recurse in that half
    sub = problem.getSubproblemContaining(subproblems, neighbor)
    if not trace is None: trace.setProblemDimensions(sub)
    result = algorithm1(sub, trace)
    return problem.getLocationInSelf(sub, result)

In [ ]:
# 2D Peak Finding Algorithm - Correct Divide and Conquer
def algorithm2(problem, location = (0, 0), trace = None):
    # if it's empty, we're done 
    if problem.numRow <= 0 or problem.numCol <= 0:
        return None

    nextLocation = problem.getBetterNeighbor(location, trace)

    if nextLocation == location:
        # there is no better neighbor, so return this peak
        if not trace is None: trace.foundPeak(location)
        return location
    else:
        # there is a better neighbor, so move to the neighbor and recurse
        return algorithm2(problem, nextLocation, trace)

In [ ]:
# 2D Peak Finding Algorithm - INCORRECT attempt
# This algorithm faultily traverses the matrix
# For clearer visualization, run the algorithm with debugger using provided counterexample
def algorithm3(problem, bestSeen = None, trace = None):
    # if it's empty, we're done 
    if problem.numRow <= 0 or problem.numCol <= 0:
        return None

    midRow = problem.numRow // 2
    midCol = problem.numCol // 2

    # first, get the list of all subproblems
    subproblems = []

    (subStartR1, subNumR1) = (0, midRow)
    (subStartR2, subNumR2) = (midRow + 1, problem.numRow - (midRow + 1))
    (subStartC1, subNumC1) = (0, midCol)
    (subStartC2, subNumC2) = (midCol + 1, problem.numCol - (midCol + 1))

    subproblems.append((subStartR1, subStartC1, subNumR1, subNumC1))
    subproblems.append((subStartR1, subStartC2, subNumR1, subNumC2))
    subproblems.append((subStartR2, subStartC1, subNumR2, subNumC1))
    subproblems.append((subStartR2, subStartC2, subNumR2, subNumC2))

    # find the best location on the cross (the middle row combined with the
    # middle column)
    cross = []
    
    cross.extend(crossProduct([midRow], range(problem.numCol)))
    cross.extend(crossProduct(range(problem.numRow), [midCol]))

    crossLoc = problem.getMaximum(cross, trace)
    neighbor = problem.getBetterNeighbor(crossLoc, trace)

    # update the best we've seen so far based on this new maximum
    if bestSeen is None or problem.get(neighbor) > problem.get(bestSeen):
        bestSeen = neighbor
        if not trace is None: trace.setBestSeen(bestSeen)

    # return if we can't see any better neighbors
    if neighbor == crossLoc:
        if not trace is None: trace.foundPeak(crossLoc)
        return crossLoc

    # figure out which subproblem contains the largest number we've seen so
    # far, and recurse
    sub = problem.getSubproblemContaining(subproblems, bestSeen)
    newBest = sub.getLocationInSelf(problem, bestSeen)
    if not trace is None: trace.setProblemDimensions(sub)
    result = algorithm3(sub, newBest, trace)
    return problem.getLocationInSelf(sub, result)

In [ ]:
# 2D Peak Finding Algorithm - Algorithm that alternates between splitting on rows and splitting on columns
# first splits on rows, then columns, then rows, ...
def algorithm4(problem, bestSeen = None, rowSplit = True, trace = None):
    # if it's empty, we're done 
    if problem.numRow <= 0 or problem.numCol <= 0:
        return None

    subproblems = []
    divider = []

    if rowSplit:
        # the recursive subproblem will involve half the number of rows
        mid = problem.numRow // 2

        # information about the two subproblems
        (subStartR1, subNumR1) = (0, mid)
        (subStartR2, subNumR2) = (mid + 1, problem.numRow - (mid + 1))
        (subStartC, subNumC) = (0, problem.numCol)

        subproblems.append((subStartR1, subStartC, subNumR1, subNumC))
        subproblems.append((subStartR2, subStartC, subNumR2, subNumC))

        # get a list of all locations in the dividing column
        divider = crossProduct([mid], range(problem.numCol))
    else:
        # the recursive subproblem will involve half the number of columns
        mid = problem.numCol // 2

        # information about the two subproblems
        (subStartR, subNumR) = (0, problem.numRow)
        (subStartC1, subNumC1) = (0, mid)
        (subStartC2, subNumC2) = (mid + 1, problem.numCol - (mid + 1))

        subproblems.append((subStartR, subStartC1, subNumR, subNumC1))
        subproblems.append((subStartR, subStartC2, subNumR, subNumC2))

        # get a list of all locations in the dividing column
        divider = crossProduct(range(problem.numRow), [mid])

    # find the maximum in the dividing row or column
    bestLoc = problem.getMaximum(divider, trace)
    neighbor = problem.getBetterNeighbor(bestLoc, trace)

    # update the best we've seen so far based on this new maximum
    if bestSeen is None or problem.get(neighbor) > problem.get(bestSeen):
        bestSeen = neighbor
        if not trace is None: trace.setBestSeen(bestSeen)

    # return when we know we've found a peak
    if neighbor == bestLoc and problem.get(bestLoc) >= problem.get(bestSeen):
        if not trace is None: trace.foundPeak(bestLoc)
        return bestLoc

    # figure out which subproblem contains the largest number we've seen so
    # far, and recurse, alternating between splitting on rows and splitting
    # on columns
    sub = problem.getSubproblemContaining(subproblems, bestSeen)
    newBest = sub.getLocationInSelf(problem, bestSeen)
    if not trace is None: trace.setProblemDimensions(sub)
    result = algorithm4(sub, newBest, not rowSplit, trace)
    return problem.getLocationInSelf(sub, result)